# Epidemiology of Sedation in Mechanical Ventilation

## Init

In [1]:
import os
os.chdir("..")
os.getcwd()

'/Users/wliao0504/code/clif/CLIF-epi-of-sedation'

### Import

In [2]:
from importlib import reload
import pandas as pd
import duckdb
from utils import pyCLIF as pc
reload(pc)
from utils.waterfall import process_resp_support_waterfall
import ipytest
import tableone
from utils.data_cleaner import remove_outliers_with_timing
import datetime
import warnings

Loaded configuration from config.json
Loaded configuration from config.json


In [ ]:
helper = pc.load_config()
site = helper['site_name'].lower()
print(f"your site name is: {site}")
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

Loaded configuration from config.json
your site name is: ucmc


## Cohort Identification

In [4]:
adt = pc.load_data("clif_adt")
hospitalization = pc.load_data("clif_hospitalization")

Data loaded successfully from /Users/wliao0504/code/clif/ucmc-clif-data/clif_adt.parquet
Data loaded successfully from /Users/wliao0504/code/clif/ucmc-clif-data/clif_hospitalization.parquet


### Create ICU-stay level unique id

In [5]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    stitched_encounters = pc.stitch_encounters(hospitalization, adt)

In [6]:
# create a mapping table
query = """
SELECT DISTINCT patient_id, hospitalization_id, encounter_block
FROM stitched_encounters
"""
hosp_to_enc_blk_mapper = duckdb.sql(query).to_df()

In [7]:
query = """
SELECT hospitalization_id
    , encounter_block
    , date_trunc('hour', in_dttm) as in_date_hr
    , 1 as new_icu_stay
FROM stitched_encounters
WHERE location_category = 'icu'
"""
new_icu_start_hours = duckdb.sql(query).to_df()

hosp_ids_w_icu_stays = new_icu_start_hours['hospitalization_id'].unique().tolist()

### Hour 24 and 72

In [8]:
resp = pc.load_data(
    table = "clif_respiratory_support",
    filters = {
        "hospitalization_id": hosp_ids_w_icu_stays
    }
    )

Data loaded successfully from /Users/wliao0504/code/clif/ucmc-clif-data/clif_respiratory_support.parquet


In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    resp_f = process_resp_support_waterfall(resp)
    resp_f.to_parquet(f"output/intermediate/{site}_resp_processed.parquet")

# resp_f = pd.read_parquet(f"output/intermediate/{site}_resp_processed.parquet")

In [10]:
# for MIMIC dev
focal_hosp_ids = [
    '21738444', 
    '20004088', 
    '20006154', 
    '20018306'
    ]

# query = f"""
# SELECT SUM(CASE WHEN location_category = 'icu' THEN 1 ELSE 0 END) as total_icu_stays
# FROM adt
# WHERE hospitalization_id IN ({",".join(focal_hosp_ids)})
# """
# duckdb.sql(query).to_df()

In [11]:
resp_f['date_hr'] = resp_f['recorded_dttm'].dt.floor('h')

query = f"""
SELECT t1.hospitalization_id
    , t3.encounter_block
    , t1.date_hr
    , MAX(CASE WHEN t2.new_icu_stay = 1 THEN 1 ELSE 0 END) as new_icu_start_from_adt
    , MAX(CASE WHEN t1.device_category = 'imv' THEN 1 ELSE 0 END) as on_imv
    , MAX(CASE WHEN t1.tracheostomy is True OR t1.tracheostomy = 1 THEN 1 ELSE 0 END) as trach_ever
    , ROW_NUMBER() OVER (PARTITION BY t1.hospitalization_id ORDER BY t1.date_hr) as rn_by_hosp
    , CASE WHEN (
        rn_by_hosp = 1 -- new hospitalization
        OR new_icu_start_from_adt = 1 -- new icu stay
    ) THEN 1 ELSE 0 END as new_icu_stay
FROM resp_f as t1
LEFT JOIN new_icu_start_hours AS t2
    ON t1.hospitalization_id = t2.hospitalization_id
    AND t1.date_hr = t2.in_date_hr
LEFT JOIN hosp_to_enc_blk_mapper AS t3
    ON t1.hospitalization_id = t3.hospitalization_id
-- WHERE t1.hospitalization_id IN ({",".join(focal_hosp_ids)})
GROUP BY t1.hospitalization_id, t1.date_hr, t3.encounter_block
ORDER BY t1.hospitalization_id, t1.date_hr
"""
df1 = duckdb.sql(query).to_df()

In [12]:
query = """
-- generate unique icu stay ids
WITH t1 AS (
    SELECT hospitalization_id
        , encounter_block
        , date_hr
        , on_imv
        , new_icu_stay
        , SUM(new_icu_stay) OVER (ORDER BY hospitalization_id, date_hr) as icu_stay_id
    FROM df1
    -- keep only hospitalizations that have at least one hour on imv and no tracheostomy
    WHERE hospitalization_id IN (
        SELECT DISTINCT hospitalization_id
        FROM df1
        GROUP BY hospitalization_id
        HAVING MAX(on_imv) = 1 AND MAX(trach_ever) = 0
    )
),
-- generate unique imv streak ids
t2 AS (
    SELECT hospitalization_id
        , icu_stay_id
        -- , encounter_block
        , date_hr
        , on_imv
        , ROW_NUMBER() OVER (PARTITION BY icu_stay_id ORDER BY date_hr) as rn_overall
        , ROW_NUMBER() OVER (PARTITION BY icu_stay_id, on_imv ORDER BY date_hr) as rn_by_imv_status
        , rn_overall - rn_by_imv_status as imv_streak_id
    FROM t1
    -- keep only icu stays that have at least one hour on imv
    WHERE icu_stay_id IN (
        SELECT DISTINCT icu_stay_id
        FROM t1
        GROUP BY icu_stay_id
        HAVING MAX(on_imv) = 1
    )
    ORDER BY hospitalization_id, icu_stay_id, date_hr
),
-- mark the 24th and 72th hour of each imv streak
t3 AS (
    SELECT hospitalization_id, icu_stay_id, date_hr
        , imv_streak_id, on_imv
        , SUM(on_imv) OVER (PARTITION BY icu_stay_id, imv_streak_id ORDER BY date_hr) as imv_hrs_in_streak
        , CASE WHEN (imv_hrs_in_streak = 24) THEN 1 ELSE 0 END as hr_24_on_imv
        , CASE WHEN (imv_hrs_in_streak = 72) THEN 1 ELSE 0 END as hr_72_on_imv
        -- calculate hour since first intubation within each icu stay
        , MIN(CASE WHEN on_imv = 1 THEN date_hr END) OVER (PARTITION BY icu_stay_id) as first_imv_hr_in_icu_stay
        -- can only calculate diff in secs, so convert to hrs
        ,  EXTRACT(EPOCH FROM (date_hr - first_imv_hr_in_icu_stay)) / 3600 + 1 as hrs_since_first_imv
    FROM t2
    ORDER BY hospitalization_id, icu_stay_id, date_hr
    )
-- exclude cases with reintubation within 72 hours
SELECT hospitalization_id, icu_stay_id, date_hr
    , imv_streak_id, on_imv, imv_hrs_in_streak, hrs_since_first_imv
    , hr_24_on_imv, hr_72_on_imv
    , COUNT(DISTINCT CASE WHEN hrs_since_first_imv BETWEEN 0 AND 72 THEN imv_streak_id END) 
        OVER (PARTITION BY icu_stay_id) as n_imv_streaks_in_72_hrs
    , CASE WHEN n_imv_streaks_in_72_hrs <= 2 AND hr_24_on_imv = 1 THEN 1 ELSE 0 END as hr_24_on_imv_noreintub
    , CASE WHEN n_imv_streaks_in_72_hrs = 1 AND hr_72_on_imv = 1 THEN 1 ELSE 0 END as hr_72_on_imv_noreintub
FROM t3
ORDER BY hospitalization_id, icu_stay_id, date_hr
"""
df2 = duckdb.sql(query).to_df()

In [13]:
# %%ipytest

# # sanity tests against the MIMIC-IV data
# @pytest.mark.parametrize("hospitalization_id,date_hr,expected_hr,expected_flag,expected_flag_noreintub", [
#     # on imv for 24-hrs twice during the same hospitalization -- so would be excluded if no reintubation within 72 hrs
#     (21738444, "2186-09-14 17:00:00-06:00", 24, 1, 0),  
#     (21738444, "2186-09-14 18:00:00-06:00", 24, 0, 0),  
#     (21738444, "2186-09-16 18:00:00-06:00", 24, 1, 0), # second streak within the hosp
#     # not on imv for the first few hrs but long streak afterwards
#     (20004088, "2159-09-30 09:00:00-06:00", 24, 1, 1),
#     (20004088, "2159-10-02 09:00:00-06:00", 72, 1, 1),
#     # very short streaks: 20006154
#     # 3 icu stays within the same hospitalization
#     (20018306, "2136-05-16 05:00:00-06:00", 24, 1, 1),
#     (20018306, "2136-07-01 19:00:00-06:00", 24, 1, 1),
#     # (20018306, "2136-06-01 03:00:00-06:00", 24, 0), # in a icu stay that was filtered out in the df because of no imv ever
# ])
# def test_if_on_imv_at_hr_x(hospitalization_id, date_hr, expected_hr, expected_flag, expected_flag_noreintub):
#     query = f"""
#     SELECT hr_{expected_hr}_on_imv, hr_{expected_hr}_on_imv_noreintub
#     FROM df2
#     WHERE hospitalization_id = {hospitalization_id}
#     AND date_hr = '{date_hr}'
#     """
#     result = duckdb.sql(query).to_df()
#     observed_flag = result[f'hr_{expected_hr}_on_imv'].iloc[0]
#     observed_flag_noreintub = result[f'hr_{expected_hr}_on_imv_noreintub'].iloc[0]
#     assert observed_flag == expected_flag
#     assert observed_flag_noreintub == expected_flag_noreintub

### The Cohort

In [14]:
# keep the cohort
query = """
SELECT ROW_NUMBER() OVER () as row_id
    , hospitalization_id
    --, encounter_block
    --, icu_stay_id
    , date_hr
    , CASE WHEN hr_24_on_imv_noreintub = 1 THEN 'hr_24'
        WHEN hr_72_on_imv_noreintub = 1 THEN 'hr_72'
        ELSE NULL END as cohort_flag
FROM df2
LEFT JOIN hosp_to_enc_blk_mapper USING (hospitalization_id)
WHERE hr_24_on_imv_noreintub = 1 OR hr_72_on_imv_noreintub = 1
"""
cohort = duckdb.sql(query).to_df()

cohort_hosp_ids = cohort['hospitalization_id'].unique().tolist()

## Demographics

In [15]:
hosp_required_columns = [
    "patient_id", 
    "hospitalization_id", 
    "age_at_admission"
]

cohort_hosp = pc.load_data(
    table = "clif_hospitalization",
    columns = hosp_required_columns,
    filters = {
        "hospitalization_id": cohort_hosp_ids
    }
)

cohort_pt_ids = cohort_hosp['patient_id'].unique().tolist()

pt_required_columns = [
    "patient_id",
    "race_category",
    "ethnicity_category",
    "sex_category"
]

cohort_pt = pc.load_data(
    table = "clif_patient",
    columns = pt_required_columns,
    filters = {
        "patient_id": cohort_pt_ids
    }
)

Data loaded successfully from /Users/wliao0504/code/clif/ucmc-clif-data/clif_hospitalization.parquet
Data loaded successfully from /Users/wliao0504/code/clif/ucmc-clif-data/clif_patient.parquet


In [16]:
query = """
SELECT *
FROM cohort_hosp
LEFT JOIN cohort_pt
USING (patient_id)
"""
cohort_demogs = duckdb.sql(query).to_df()

## Vitals

In [17]:
vitals_required_columns = [
    "hospitalization_id",
    "recorded_dttm",
    "vital_category",
    "vital_value"
]

vitals = pc.load_data(
    table = "clif_vitals",
    filters = {
        "hospitalization_id": cohort_hosp_ids
    },
    columns = vitals_required_columns
)

vitals['date_hr'] = vitals['recorded_dttm'].dt.floor('h')

Data loaded successfully from /Users/wliao0504/code/clif/ucmc-clif-data/clif_vitals.parquet


In [18]:
vitals = remove_outliers_with_timing(vitals, 'vitals', 'vital_value', file_path='config/outliers.json')

Outliers found in vitals
Category 'map': 5022 outliers (0.39%) have been set to NaN.
Category 'spo2': 2081 outliers (0.08%) have been set to NaN.
Category 'temp_c': 1667 outliers (0.15%) have been set to NaN.
Category 'respiratory_rate': 641 outliers (0.02%) have been set to NaN.
Category 'sbp': 60 outliers (0.00%) have been set to NaN.
Category 'height_cm': 24 outliers (0.09%) have been set to NaN.
Category 'weight_kg': 21 outliers (0.02%) have been set to NaN.
Category 'heart_rate': 3 outliers (0.00%) have been set to NaN.
Category 'dbp': 3 outliers (0.00%) have been set to NaN.
⏱️ Outlier removal completed in 7.13 seconds
🟢 Processed 16,783,169 rows


In [19]:
query = f"""
SELECT *
FROM cohort c
CROSS JOIN (SELECT DISTINCT vital_category FROM vitals) v
ORDER BY hospitalization_id, date_hr, vital_category
"""
cohort_hrs_cross_vital_categories = duckdb.sql(query).to_df()

In [20]:
query = """
-- fill any missing values in the cohort hours with the nearest in time (in the past or future)
SELECT hospitalization_id
    , date_hr
    , cohort_flag
    , vital_category
    , MEAN(vital_value) as mean_value
    
    , LAG(mean_value) OVER (PARTITION BY hospitalization_id, vital_category ORDER BY date_hr) as mean_value_lag
    , LAST_VALUE(mean_value IGNORE NULLS) OVER (
        PARTITION BY hospitalization_id, vital_category 
        ORDER BY date_hr
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) as mean_value_final
    -- , COALESCE(mean_value, mean_value_last) as mean_value_final
FROM cohort_hrs_cross_vital_categories
FULL OUTER JOIN vitals USING (hospitalization_id, date_hr, vital_category)
GROUP BY hospitalization_id, date_hr, cohort_flag, vital_category
ORDER BY hospitalization_id, vital_category, date_hr, cohort_flag
"""
vitals_hrly = duckdb.sql(query).to_df()

In [21]:
query = """
SELECT hospitalization_id
    , date_hr
    , cohort_flag
    , vital_category
    , mean_value
    , mean_value_final
FROM vitals_hrly
WHERE cohort_flag IS NOT NULL
ORDER BY hospitalization_id, vital_category, date_hr, cohort_flag
"""
vitals_cohort_hrs = duckdb.sql(query).to_df()

In [22]:
vitals_cohort_hrs_w = vitals_cohort_hrs.pivot_table(
    index=['hospitalization_id', 'date_hr', 'cohort_flag'], 
    columns='vital_category', 
    values='mean_value_final', 
    fill_value=0
).reset_index()
vitals_cohort_hrs_w.columns.name = None

## Assessments

In [23]:
pa_required_columns = [
    "hospitalization_id", 
    "recorded_dttm",
    "assessment_category", 
    "numerical_value"
]

pa = pc.load_data(
    table = "clif_patient_assessments",
    columns = pa_required_columns,
    filters = {
        "hospitalization_id": cohort_hosp_ids,
        "assessment_category": ["gcs_total", "rass", "RASS"]
    }
)

pa['date_hr'] = pa['recorded_dttm'].dt.floor('h')

Data loaded successfully from /Users/wliao0504/code/clif/ucmc-clif-data/clif_patient_assessments.parquet


In [24]:
# pa_required_columns = [
#     "hospitalization_id", 
#     "recorded_dttm",
#     "assessment_category", 
#     "numerical_value"
# ]

# gcs = pc.load_data(
#     table = "clif_patient_assessments",
#     columns = pa_required_columns,
#     filters = {
#         "hospitalization_id": cohort_hosp_ids,
#         "assessment_category": ["gcs_total", #"rass", "RASS", 
#                                 "gcs_verbal", "gcs_motor", "gcs_eye"]
#     }
# )

# gcs['date_hr'] = gcs['recorded_dttm'].dt.floor('h')

# gcs_w = gcs.pivot_table(
#     index=['hospitalization_id', 'recorded_dttm'],
#     columns='assessment_category',
#     values='numerical_value',
#     aggfunc='mean'
# ).reset_index()

# # Flatten column names
# gcs_w.columns.name = None

### Intubaton start hour

In [25]:
query = """
SELECT *
    -- note that we subtract 23 instead of 24 to be inclusive of all minutes in the starting hr
    , CASE WHEN cohort_flag = 'hr_24' THEN date_hr - INTERVAL '20 hours' 
        WHEN cohort_flag = 'hr_72' THEN date_hr - INTERVAL '68 hours'
        ELSE NULL END as date_hr_intub
FROM cohort c
"""
cohort_intub = duckdb.sql(query).to_df()

### Most recent GCS

In [26]:
def find_most_recent_gcs(target_dttm_name: str = 'date_hr_intub', window_in_hr: int = 22):
    '''
    Find the most recent GCS assessment to the target time.
    '''
    query = f"""
    WITH t1 AS (
        SELECT *
            , CASE WHEN cohort_flag = 'hr_24' THEN date_hr - INTERVAL '{window_in_hr} hours' 
                WHEN cohort_flag = 'hr_72' THEN date_hr - INTERVAL '{window_in_hr + 48} hours'
                ELSE NULL END as date_hr_intub
        FROM cohort c
    )
    SELECT c.*
        , p.numerical_value as gcs_total
        , p.recorded_dttm as gcs_recorded_dttm
        -- rn = 1 for the gcs w/ the latest recorded_dttm (and thus most recent)
        , ROW_NUMBER() OVER (
            PARTITION BY c.row_id, c.hospitalization_id, c.{target_dttm_name}
            ORDER BY p.recorded_dttm DESC
            ) as rn
    FROM t1 c
    LEFT JOIN pa p
        ON c.hospitalization_id = p.hospitalization_id 
        AND p.assessment_category = 'gcs_total'
        AND p.numerical_value IS NOT NULL
        AND p.recorded_dttm <= c.{target_dttm_name}
    QUALIFY (rn = 1) -- OR (gcs_total IS NULL) -- include cohort even if no gcs found
    ORDER BY row_id, c.{target_dttm_name}, p.recorded_dttm
    """
    return duckdb.sql(query).to_df()

pa_gcs_intub_hr = find_most_recent_gcs(target_dttm_name='date_hr_intub')
pa_gcs_cohort_hr = find_most_recent_gcs(target_dttm_name='date_hr')


### Most recent RASS

In [27]:
def find_most_recent(clif_category: str, hr_from_cohort_dttm: int = 1):
    '''
    Find the most recent assessment to the target time.
    '''
    query = f"""
    WITH t1 AS (
        SELECT *
            , CASE WHEN cohort_flag = 'hr_24' THEN date_hr - INTERVAL '{hr_from_cohort_dttm} hours' 
                WHEN cohort_flag = 'hr_72' THEN date_hr - INTERVAL '{hr_from_cohort_dttm + 48} hours'
                ELSE NULL END as date_hr_intub
            , date_hr + INTERVAL '{hr_from_cohort_dttm} hours' as target_dttm
        FROM cohort c
    )
    SELECT c.*
        , p.numerical_value as value
        , p.recorded_dttm
        -- rn = 1 for the gcs w/ the latest recorded_dttm (and thus most recent)
        , ROW_NUMBER() OVER (
            PARTITION BY c.row_id, c.hospitalization_id, c.target_dttm
            ORDER BY p.recorded_dttm DESC
            ) as rn
    FROM t1 c
    LEFT JOIN pa p
        ON c.hospitalization_id = p.hospitalization_id 
        AND p.assessment_category in ('{clif_category}', '{clif_category.lower()}', '{clif_category.upper()}')
        AND p.numerical_value IS NOT NULL
        AND p.recorded_dttm <= c.target_dttm
    QUALIFY (rn = 1) --OR (value IS NULL) -- include cohort even if no clif_category found
    ORDER BY row_id, c.target_dttm, p.recorded_dttm
    """
    return duckdb.sql(query).to_df()

cohort_rass = find_most_recent(clif_category='rass', hr_from_cohort_dttm=1)


## Medication

In [28]:
sed_med_categories = [
    "midazolam", "lorazepam", "hydromorphone", "fentanyl", "propofol", "dexmedetomidine", "ketamine"
]

vaso_med_categories = [
    "epinephrine", "norepinephrine", "phenylephrine", "vasopressin", "angiotensin", 
    "dopamine", "dobutamine"
]

mac_required_columns = [
    "hospitalization_id", 
    "admin_dttm",
    "med_category",
    "med_group",
    "med_dose",
    "med_dose_unit",
    "mar_action_name"
]

mac = pc.load_data(
    table = "clif_medication_admin_continuous",
    columns = mac_required_columns,
    filters = {
        "hospitalization_id": cohort_hosp_ids,
        "med_category": sed_med_categories + vaso_med_categories
    }
)

mac['date_hr'] = mac['admin_dttm'].dt.floor('h')

Data loaded successfully from /Users/wliao0504/code/clif/ucmc-clif-data/clif_medication_admin_continuous.parquet


In [ ]:
# # Pivot MAC to wide format
# mac_pivot = mac.pivot_table(
#     index=['hospitalization_id', 'admin_dttm'],
#     columns='med_category',
#     values='med_dose',
#     # aggfunc='sum',
#     fill_value=0
# ).reset_index()

# # Flatten column names
# mac_pivot.columns.name = None

### Check dosage unit

In [ ]:
query = """
SELECT med_category, med_dose_unit, COUNT(*) as n
FROM mac
GROUP BY med_category, med_dose_unit
ORDER BY med_category, n DESC
"""
med_units_count = duckdb.sql(query).to_df()
med_units_count.to_csv(f"output/final/{site}_med_units_count_{timestamp}.csv")

### Most recent patient weight

In [31]:
query = """
SELECT m.*
    , v.vital_value as weight_kg
    , v.recorded_dttm as weight_recorded_dttm
    -- rn = 1 for the weight w/ the latest recorded_dttm (and thus most recent)
    , ROW_NUMBER() OVER (
        PARTITION BY m.hospitalization_id, m.admin_dttm, m.med_category
        ORDER BY v.recorded_dttm DESC
        ) as rn
FROM mac m
LEFT JOIN vitals v 
    ON m.hospitalization_id = v.hospitalization_id 
    AND v.vital_category = 'weight_kg' AND v.vital_value IS NOT NULL
    AND v.recorded_dttm <= m.admin_dttm  -- only past weights
QUALIFY (rn = 1) -- OR (weight_kg IS NULL) -- include meds even if no weight found
ORDER BY m.hospitalization_id, m.admin_dttm, m.med_category, rn
"""
mac_w_wt = duckdb.sql(query).to_df()

### Standardize dosage unit

In [32]:
def standardize_dose_unit(df_name: str) -> pd.DataFrame:
    """
    Standardize everything to mcg/min.
    Assumes the presentation of the following columns:
    - med_dose_unit: the original unit of the dose
    - med_dose: the original dose
    - weight_kg: the (imputed, most recent) weight of the patient
    """
    query = f"""
    SELECT *
        , LOWER(med_dose_unit) AS med_dose_unit_lower
        , CASE WHEN regexp_matches(med_dose_unit_lower, '/h(r|our)?\\b') THEN 1/60.0
            WHEN regexp_matches(med_dose_unit_lower, '/m(in|inute)?\\b') THEN 1.0
            ELSE NULL END as time_multiplier
        , CASE WHEN contains(med_dose_unit_lower, '/kg/') THEN weight_kg
            ELSE 1 END AS pt_weight_adjustment
        , CASE WHEN contains(med_dose_unit_lower, 'mcg/') THEN 1.0
            WHEN contains(med_dose_unit_lower, 'mg/') THEN 1000.0
            WHEN contains(med_dose_unit_lower, 'ng/') THEN 0.001
            WHEN contains(med_dose_unit_lower, 'milli') THEN 0.001
            ELSE NULL END as dose_mass_multiplier
        , med_dose * time_multiplier * pt_weight_adjustment * dose_mass_multiplier as med_dose_converted
        , CASE WHEN contains(med_dose_unit_lower, 'units/') THEN 'units/min'
            ELSE 'mcg/min' END as med_dose_unit_converted
    FROM {df_name}
    """
    return duckdb.sql(query).to_df()

mac_converted = standardize_dose_unit('mac_w_wt')

In [33]:
mac_converted.value_counts('mar_action_name')

mar_action_name
Rate Verify                       1199722
Rate Change                        312265
New Bag                            223261
Stopped                             59165
Canceled Entry                      22776
Restarted                           20642
Held                                 7773
Paused                               4753
Missed                               2569
Unheld by Provider                   1706
Held by Provider                     1694
Due                                  1138
Completed                             686
Given                                 654
Bolus                                 533
Rate  Change                          164
Automatically Held                     82
Given by Other                         70
New  Bag                               28
Pending                                28
See Paper Documentation                16
Refused                                15
Restart Infusion                       14
Stop Infusion     

### Remove duplicates

In [34]:
mac_converted.drop_duplicates(
    subset=['hospitalization_id', 'admin_dttm', 'med_category', 'med_dose_converted'], 
    inplace=True
    )

def remove_meds_duplicates(meds_df_name: str) -> pd.DataFrame:
    query = f"""
    SELECT *
        , LOWER(mar_action_name) as mar_action_lower
        , ROW_NUMBER() OVER (
            PARTITION BY hospitalization_id, admin_dttm, med_category
            ORDER BY 
                CASE
                    WHEN contains(mar_action_lower, 'verify') THEN 10 -- CAST('inf' AS DOUBLE)
                    -- WHEN contains(mar_action_lower, 'stopped') THEN 9
                    ELSE 1
                END,
                CASE -- deprioritize zero or null doses
                    WHEN med_dose > 0 THEN 1
                    ELSE 2
                END,
                med_dose desc -- prioritize larger doses
        ) as rn_dedup
    FROM {meds_df_name}
    -- QUALIFY rn_dedup = 1
    ORDER BY hospitalization_id, admin_dttm, med_category;
    """
    return duckdb.sql(query).to_df()
    
mac_deduped = remove_meds_duplicates('mac_converted')

### Scaffold cohort with med_categories

In [35]:
query = f"""
SELECT *
FROM cohort c
CROSS JOIN (SELECT DISTINCT med_category FROM mac) m
ORDER BY hospitalization_id, date_hr, med_category
"""
cohort_hrs_cross_med_categories = duckdb.sql(query).to_df()

### Calculate cumulative dosage

In [36]:
query = """
-- insert the cohort hours into the mac record
SELECT hospitalization_id
    , cohort_flag
    , med_category
    , date_hr
    , admin_dttm
    , mar_action_name
    -- , med_dose, med_dose_unit
    , med_dose_converted as med_dose
    , med_dose_unit_converted as med_dose_unit
    , weight_kg
    , LAST_VALUE(admin_dttm IGNORE NULLS) OVER (
        PARTITION BY hospitalization_id, med_category 
        ORDER BY date_hr, admin_dttm 
        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ) as admin_dttm_last
    , LAST_VALUE(med_dose IGNORE NULLS) OVER (
        PARTITION BY hospitalization_id, med_category 
        ORDER BY date_hr, admin_dttm 
        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ) as med_dose_last
    , LAST_VALUE(mar_action_name IGNORE NULLS) OVER (
        PARTITION BY hospitalization_id, med_category 
        ORDER BY date_hr, admin_dttm 
        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ) as mar_action_name_last
    -- add helper flags for first and last streak of the same med within the hour -- which needs special handling
    , CASE WHEN admin_dttm = MIN(admin_dttm) OVER (PARTITION BY hospitalization_id, med_category, date_hr) 
        THEN 1 ELSE 0 END as is_first_streak
    , CASE WHEN admin_dttm = MAX(admin_dttm) OVER (PARTITION BY hospitalization_id, med_category, date_hr) 
        THEN 1 ELSE 0 END as is_last_streak
    , date_hr + INTERVAL '1 hour' as date_hr_next
FROM cohort_hrs_cross_med_categories c
FULL OUTER JOIN mac_deduped m USING (hospitalization_id, date_hr, med_category)
ORDER BY hospitalization_id, med_category, date_hr, admin_dttm
"""
mac_hrly = duckdb.sql(query).to_df()

In [37]:
# query = """
# SELECT *

# FROM mac_hrly
# ORDER BY hospitalization_id, med_category, date_hr, admin_dttm
# """
# # forward filled
# mac_hrly_ff = duckdb.sql(query).to_df()

In [38]:
query = """
-- keep only the med admins that are within the cohort hours
SELECT *
FROM mac_hrly
WHERE cohort_flag IS NOT NULL -- AND admin_dttm_last IS NOT NULL
ORDER BY hospitalization_id, med_category, date_hr, admin_dttm
"""
mac_cohort_hrs = duckdb.sql(query).to_df()

In [39]:
mask = mac_cohort_hrs['cohort_flag'] == 'hr_24'
mask.sum()

np.int64(139217)

In [40]:
query = f"""
-- calculate the cumulative dosage within the hr
SELECT hospitalization_id, cohort_flag, med_category, date_hr
    , SUM(CASE 
        -- if no mac admin record within the cohort hour, use the last observed dose and assume it runs the entire 60 mins
        WHEN admin_dttm IS NULL
            THEN 60.0 * COALESCE(med_dose_last, 0)
        -- otherwise, calculate the cumulative dosage within the hr
        WHEN is_first_streak = 1
            THEN EXTRACT(EPOCH FROM (admin_dttm - date_hr))/60.0 * med_dose_last 
        WHEN is_first_streak != 1 AND admin_dttm IS NOT NULL
            THEN EXTRACT(EPOCH FROM (admin_dttm - admin_dttm_last))/60.0 * med_dose_last 
        WHEN is_last_streak = 1
            THEN EXTRACT(EPOCH FROM (date_hr_next - admin_dttm))/60.0 * med_dose
        ELSE 0 END) as total_dosage
    , COUNT(DISTINCT CASE 
        WHEN total_dosage > 0 AND med_category in {vaso_med_categories}
        THEN med_category END) OVER (PARTITION BY hospitalization_id, date_hr) as n_pressors
FROM mac_cohort_hrs
GROUP BY hospitalization_id, med_category, date_hr, cohort_flag, med_dose_unit
ORDER BY hospitalization_id, med_category, date_hr
"""
mac_cohort_dosage = duckdb.sql(query).to_df()

In [41]:
mac_cohort_dosage_w = mac_cohort_dosage.pivot_table(
    index=['hospitalization_id', 'date_hr', 'cohort_flag', 'n_pressors'], 
    columns='med_category', 
    values='total_dosage', 
    fill_value=0
).reset_index()
mac_cohort_dosage_w.columns.name = None

In [42]:
# query = """
# SELECT hospitalization_id
#     , date_hr
#     , cohort_flag
#     , med_category
#     , COALESCE(total_dosage, 0) as total_dosage
#     , med_dose_unit
# FROM cohort_hrs_cross_med_categories c
# FULL OUTER JOIN mac_cohort_dosage m USING (hospitalization_id, date_hr, med_category, cohort_flag)
# ORDER BY hospitalization_id, cohort_flag, med_category, date_hr
# """
# df = duckdb.sql(query).to_df()

In [43]:
mac_cohort_dosage['med_category'].unique().tolist()

['angiotensin',
 'dexmedetomidine',
 'dobutamine',
 'dopamine',
 'epinephrine',
 'fentanyl',
 'hydromorphone',
 'ketamine',
 'lorazepam',
 'midazolam',
 'norepinephrine',
 'phenylephrine',
 'propofol',
 'vasopressin']

## SOFA score

In [44]:
query = """
SELECT row_id
    , date_hr - INTERVAL '23 hours' as start_dttm
    , date_hr + INTERVAL '1 hours'as stop_dttm
FROM cohort
"""
sofa_input = duckdb.sql(query).to_df()

In [45]:
query = """
SELECT row_id
    , hospitalization_id
FROM cohort
"""
id_mappings = duckdb.sql(query).to_df()

In [46]:
from utils import sofa_score
reload(sofa_score)

Loaded configuration from config.json


<module 'utils.sofa_score' from '/Users/wliao0504/code/clif/CLIF-epi-of-sedation/utils/sofa_score.py'>

In [47]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    sofa_score = sofa_score.compute_sofa(
        ids_w_dttm=sofa_input,
        tables_path=helper['tables_path'],
        use_hospitalization_id=False,
        id_mapping=id_mappings,
        output_filepath=f"output/intermediate/{site}_sofa.parquet",
        helper_module=pc
    )

2025-08-06 12:39:07,702 - INFO - Starting SOFA computation for 13435 rows
2025-08-06 12:39:08,150 - INFO - Loaded 808345 lab rows


Data loaded successfully from /Users/wliao0504/code/clif/ucmc-clif-data/clif_labs.parquet
lab_collect_dttm: null count before conversion= 0
lab_collect_dttm: Your timezone is UTC, Converting to your site timezone (US/Central).
lab_collect_dttm: null count after conversion= 0


2025-08-06 12:39:10,012 - INFO - Loaded 4071121 vitals rows


Data loaded successfully from /Users/wliao0504/code/clif/ucmc-clif-data/clif_vitals.parquet
recorded_dttm: null count before conversion= 0
recorded_dttm: Your timezone is UTC, Converting to your site timezone (US/Central).
recorded_dttm: null count after conversion= 0


2025-08-06 12:39:11,707 - INFO - Created vitals summary
2025-08-06 12:39:11,709 - INFO - Imputed pao2 from spo2
2025-08-06 12:39:11,999 - INFO - Loaded 467601 GCS rows


...

2025-08-06 12:39:15,246 - INFO - Loaded 1514037 resp rows


Data loaded successfully from /Users/wliao0504/code/clif/ucmc-clif-data/clif_respiratory_support.parquet
recorded_dttm: null count before conversion= 0
recorded_dttm: Your timezone is UTC, Converting to your site timezone (US/Central).
recorded_dttm: null count after conversion= 0
FIO2_SET mean= 0.5025205423861635 is within the required range


/Users/wliao0504/code/clif/CLIF-epi-of-sedation/utils/sofa_score.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resp_new['device_category'] = resp_new.apply(pyCLIF.categorize_device, axis=1)
/Users/wliao0504/code/clif/CLIF-epi-of-sedation/utils/sofa_score.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resp_new['fio2_combined'] = resp_new.apply(pyCLIF.refill_fio2, axis=1)
2025-08-06 12:39:21,823 - INFO - Loaded 393513 CRRT rows


Missing ratio of p_f (po2_arterial_recent / fio2_recent):  0.1797984425103069
Missing ratio of p_f_imputed (pao2_imputed_recent / fio2_recent):  0.6624675522980608
Missing ratio of s_f (spo2_recent / fio2_recent): 0.0011452130096197893

Most of the missing values in p_f_imputed are caused by pao2_imputed_recent, which is set to NaN when spo2>97
Data loaded successfully from /Users/wliao0504/code/clif/ucmc-clif-data/clif_crrt_therapy.parquet
recorded_dttm: null count before conversion= 0
recorded_dttm: Your timezone is UTC, Converting to your site timezone (US/Central).
recorded_dttm: null count after conversion= 0


2025-08-06 12:39:21,913 - INFO - Finished computing SOFA for 13098 ids
2025-08-06 12:39:21,922 - INFO - Wrote results to output/intermediate/ucmc_sofa.parquet


In [48]:
sofa_score.columns

Index(['row_id', 'fio2_recent', 'resp_support_recent', 'map_recent',
       'spo2_recent', 'pao2_imputed_recent', 'bilirubin_total_recent',
       'creatinine_recent', 'platelet_count_recent', 'po2_arterial_recent',
       'gcs_score_recent', 'angiotensin', 'dobutamine', 'dopamine',
       'epinephrine', 'norepinephrine', 'phenylephrine', 'vasopressin', 'p_f',
       'p_f_imputed', 's_f', 'sofa_cv_97', 'sofa_coag', 'sofa_liver',
       'sofa_resp_pf', 'sofa_resp_pf_imp', 'sofa_resp', 'sofa_cns',
       'sofa_renal', 'crrt_flag', 'sofa_total'],
      dtype='object')

In [49]:
sofa_score.drop(columns=vaso_med_categories, inplace=True)

## Merge results

In [50]:
vaso_med_categories

['epinephrine',
 'norepinephrine',
 'phenylephrine',
 'vasopressin',
 'angiotensin',
 'dopamine',
 'dobutamine']

In [51]:
query = f"""
SELECT row_id
    , hospitalization_id, date_hr, cohort_flag
    , EXTRACT(hour FROM date_hr) as hr
    , CASE 
        WHEN EXTRACT(hour FROM date_hr) >= 7 AND EXTRACT(hour FROM date_hr) < 19 
        THEN 'day' ELSE 'night' END AS shift
    , d.age_at_admission as age
    , d.race_category as race
    , d.ethnicity_category as ethnicity
    , d.sex_category as sex
    , v.*
    , r.value as rass
    -- sedatives
    , LEAST(m.midazolam, 60*1000) as midazolam
    , LEAST(m.lorazepam, 10*1000) as lorazepam
    , LEAST(m.hydromorphone, 3*1000) as hydromorphone
    , LEAST(m.fentanyl, 700) as fentanyl -- too low?
    , LEAST(m.propofol, 4000*v.weight_kg) as propofol
    , LEAST(m.dexmedetomidine, 1.5*v.weight_kg) as dexmedetomidine
    , LEAST(m.ketamine, 4000*v.weight_kg) as ketamine
    -- vasopressors
    , LEAST(m.norepinephrine, 3*60*v.weight_kg) as norepinephrine
    , LEAST(m.epinephrine, 0.1*60*v.weight_kg) as epinephrine
    , LEAST(m.phenylephrine, 5*60*v.weight_kg) as phenylephrine
    , LEAST(m.dopamine, 20*60*v.weight_kg) as dopamine
    , LEAST(m.dobutamine, 40*60*v.weight_kg) as dobutamine
    , LEAST(m.vasopressin, 0.1*60*v.weight_kg) as vasopressin
    , LEAST(m.angiotensin, 0.2*60*v.weight_kg) as angiotensin
    -- converted equivalents
    , hydromorphone * 0.05 + fentanyl AS fentanyl_eq
    , lorazepam * 2 + m.midazolam AS midazolam_eq
    , norepinephrine + epinephrine + phenylephrine / 10.0 + dopamine / 100.0 
        + vasopressin * 2.5 + angiotensin * 10 as ne_eq
    , n_pressors
    , s.sofa_total as sofa
    , COALESCE(s.po2_arterial_recent, s.pao2_imputed_recent) AS pao2
    , s.fio2_recent * 100 AS fio2
    , COALESCE(s.p_f, s.p_f_imputed) / 100 AS p_f
FROM cohort c
LEFT JOIN cohort_demogs d USING (hospitalization_id)
LEFT JOIN vitals_cohort_hrs_w v USING (hospitalization_id, date_hr, cohort_flag)
LEFT JOIN cohort_rass r USING (row_id)
LEFT JOIN mac_cohort_dosage_w m USING (hospitalization_id, date_hr, cohort_flag)
LEFT JOIN sofa_score s USING (row_id) 
ORDER BY row_id
"""
cohort_results = duckdb.sql(query).to_df()

cohort_results_24 = cohort_results[cohort_results['cohort_flag'] == 'hr_24'].copy()

cohort_results_72 = cohort_results[cohort_results['cohort_flag'] == 'hr_72'].copy()

In [52]:
cohort_results.to_parquet(f"output/intermediate/{site}_cohort_results.parquet")

## Table one

In [53]:
cohort_results.columns

Index(['row_id', 'hospitalization_id', 'date_hr', 'cohort_flag', 'hr', 'shift',
       'age', 'race', 'ethnicity', 'sex', 'hospitalization_id_1', 'date_hr_1',
       'cohort_flag_1', 'dbp', 'heart_rate', 'height_cm', 'map',
       'respiratory_rate', 'sbp', 'spo2', 'temp_c', 'weight_kg', 'rass',
       'midazolam', 'lorazepam', 'hydromorphone', 'fentanyl', 'propofol',
       'dexmedetomidine', 'ketamine', 'norepinephrine', 'epinephrine',
       'phenylephrine', 'dopamine', 'dobutamine', 'vasopressin', 'angiotensin',
       'fentanyl_eq', 'midazolam_eq', 'ne_eq', 'n_pressors', 'sofa', 'pao2',
       'fio2', 'p_f'],
      dtype='object')

In [54]:
vitals_vars = [
    'sbp', 'dbp', 'map', 'heart_rate', 'respiratory_rate', 'spo2', 'temp_c', 'weight_kg', 'height_cm'
    ]

resp_vars = ['pao2', 'fio2', 'p_f']

cat_vars = ['race', 'ethnicity', 'sex']
cont_vars = ['age', 'rass', 'sofa', 'fentanyl_eq', 'midazolam_eq', 'ne_eq', 'n_pressors'] \
    + resp_vars + vitals_vars + sed_med_categories + vaso_med_categories


In [ ]:
def gen_and_save_tableone(file_name, **kwargs):
    """
    Wrapper for tableone.TableOne that automatically saves results to a CSV file.
    
    Args:
        data: DataFrame to create table one from
        file_name: Name for the output file (without extension)
        **kwargs: All other arguments passed to tableone.TableOne
    
    Returns:
        tableone.TableOne object
    """
    table = tableone.TableOne(**kwargs, pval=True)
    table.to_csv(f'output/final/{site}_{file_name}_{timestamp}.csv')
    return table


### By cohort

In [56]:
gen_and_save_tableone(
    file_name='table_one_by_cohort_hr',
    data=cohort_results, 
    continuous=cont_vars, 
    categorical=cat_vars, 
    groupby='cohort_flag',
    nonnormal=['age', 'rass', 'sofa'] + resp_vars + vitals_vars
    )

Grouped by cohort_flag                                                                       
                                                                                          Missing              Overall                hr_24                hr_72 P-Value
n                                                                                                                13435                 9669                 3766        
age, median [Q1,Q3]                                                                             0     60.0 [45.0,70.0]     61.0 [46.0,70.0]     59.0 [43.0,69.0]  <0.001
race, n (%)                      American Indian or Alaska Native                                             36 (0.3)             28 (0.3)              8 (0.2)   0.185
                                 Asian                                                                       218 (1.6)            167 (1.7)             51 (1.4)        
                                 Black or African American                                                 8001 (59.6)          5718 (59.1)          2283 (60.6)        
                                 Native Hawaiian or Other Pacific Islander                                    30 (0.2)             23 (0.2)              7 (0.2)        
                                 None                                                                         44 (0.3)             32 (0.3)             12 (0.3)        
                                 Other                                                                       366 (2.7)            248 (2.6)            118 (3.1)        
                                 Unknown                                                                    1311 (9.8)            943 (9.8)            368 (9.8)        
                                 White                                                                     3429 (25.5)          2510 (26.0)           919 (24.4)        
ethnicity, n (%)                 Hispanic                                                                    918 (6.8)            636 (6.6)            282 (7.5)   0.309
                                 Non-Hispanic                                                             11542 (85.9)          8326 (86.1)          3216 (85.4)        
                                 None                                                                         47 (0.3)             34 (0.4)             13 (0.3)        
                                 Unknown                                                                     928 (6.9)            673 (7.0)            255 (6.8)        
sex, n (%)                       Female                                                                    5410 (40.3)          3912 (40.5)          1498 (39.8)   0.481
                                 Male                                                                      8025 (59.7)          5757 (59.5)          2268 (60.2)        
dbp, median [Q1,Q3]                                                                             0     65.0 [56.0,75.0]     64.0 [55.5,74.0]     67.0 [58.0,78.0]  <0.001
heart_rate, median [Q1,Q3]                                                                      0    89.0 [76.0,102.0]    89.0 [76.0,102.0]    89.0 [76.5,101.0]   0.911
height_cm, median [Q1,Q3]                                                                       0  170.2 [162.6,177.8]  170.2 [160.0,177.8]  170.2 [162.6,180.0]  <0.001
map, median [Q1,Q3]                                                                             0     77.0 [66.7,89.0]     76.0 [66.0,87.0]     79.0 [69.0,92.0]  <0.001
respiratory_rate, median [Q1,Q3]                                                                0     19.0 [16.0,23.0]     19.0 [15.8,23.0]     19.0 [16.0,23.0]   0.108
sbp, median [Q1,Q3]                                                                             0  120.0 [105.0,136.0]  118.0 [104.0,134.5]  123.5 [109.0,140.0]  <0.001
spo2, median [Q1,Q3

### By race

In [57]:
gen_and_save_tableone(
    file_name='table_one_by_race',
    data=cohort_results, 
    continuous=cont_vars, 
    categorical=['ethnicity', 'sex'], 
    groupby='race',
    nonnormal=['age', 'rass', 'sofa'] + resp_vars + vitals_vars
    )

Grouped by race                                                                                                                                                                                                                                           
                                                      Missing              Overall American Indian or Alaska Native                Asian Black or African American Native Hawaiian or Other Pacific Islander                 None                Other              Unknown                White P-Value
n                                                                            13435                               36                  218                      8001                                        30                   44                  366                 1311                 3429        
age, median [Q1,Q3]                                         0     60.0 [45.0,70.0]                 46.0 [42.8,56.0]     62.0 [49.0,72.8]          60.0 [43.0,69.0]                          54.0 [30.8,67.5]     54.5 [48.0,66.0]     51.0 [35.2,64.0]     59.0 [46.0,69.0]     63.0 [51.0,71.0]  <0.001
ethnicity, n (%)                 Hispanic                                918 (6.8)                        14 (38.9)              6 (2.8)                  32 (0.4)                                 22 (73.3)              0 (0.0)           294 (80.3)           263 (20.1)            287 (8.4)  <0.001
                                 Non-Hispanic                         11542 (85.9)                        22 (61.1)           204 (93.6)               7865 (98.3)                                  8 (26.7)              0 (0.0)            68 (18.6)           328 (25.0)          3047 (88.9)        
                                 None                                     47 (0.3)                          0 (0.0)              0 (0.0)                   2 (0.0)                                   0 (0.0)           44 (100.0)              0 (0.0)              1 (0.1)              0 (0.0)        
                                 Unknown                                 928 (6.9)                          0 (0.0)              8 (3.7)                 102 (1.3)                                   0 (0.0)              0 (0.0)              4 (1.1)           719 (54.8)             95 (2.8)        
sex, n (%)                       Female                                5410 (40.3)                        18 (50.0)            79 (36.2)               3423 (42.8)                                  6 (20.0)            15 (34.1)           126 (34.4)           445 (33.9)          1298 (37.9)  <0.001
                                 Male                                  8025 (59.7)                        18 (50.0)           139 (63.8)               4578 (57.2)                                 24 (80.0)            29 (65.9)           240 (65.6)           866 (66.1)          2131 (62.1)        
dbp, median [Q1,Q3]                                         0     65.0 [56.0,75.0]                 69.5 [62.0,76.4]     63.4 [54.0,72.4]          66.0 [57.0,77.0]                          64.7 [55.1,73.8]     68.8 [58.0,74.2]     64.0 [56.0,73.9]     64.0 [56.0,74.4]     63.0 [55.0,72.0]  <0.001
heart_rate, median [Q1,Q3]                                  0    89.0 [76.0,102.0]                 92.0 [76.8,99.2]    88.5 [77.0,100.9]         89.0 [76.0,103.0]                          90.0 [84.5,98.4]    94.0 [78.0,102.2]    87.8 [76.0,101.0]    88.7 [77.0,101.0]    88.0 [76.0,100.0]   0.018
height_cm, median [Q1,Q3]                                   0  170.2 [162.6,177.8]              167.6 [164.4,175.9]  165.1 [154.9,172.7]       170.2 [162.6,177.8]                       170.1 [163.2,172.7]  165.6 [160.6,182.9]  166.1 [158.1,175.3]  170.2 [162.6,177.8]  170.2 [162.6,180.3]  <0.001
map, median [Q1,Q3]                                         0     77.0 [66.7,89.0]                 77.2 [33.0,90.2]     77.0 [67.0,87.0]          77.0 [66.0,90.0]                       

### By shift

In [58]:
gen_and_save_tableone(
    file_name='table_one_by_shift_hr24',
    data=cohort_results_24, 
    continuous=cont_vars, 
    categorical=['race', 'ethnicity', 'sex'], 
    groupby='shift'
    # nonnormal=['age', 'rass', 'sofa'] + resp_vars + vitals_vars,    
    )

Grouped by shift                                                        
                                                                               Missing         Overall             day           night P-Value
n                                                                                                 9669            5444            4225        
age, mean (SD)                                                                       0     57.6 (17.4)     59.0 (16.6)     55.8 (18.1)  <0.001
race, n (%)                 American Indian or Alaska Native                                  28 (0.3)        17 (0.3)        11 (0.3)  <0.001
                            Asian                                                            167 (1.7)       106 (1.9)        61 (1.4)        
                            Black or African American                                      5718 (59.1)     3000 (55.1)     2718 (64.3)        
                            Native Hawaiian or Other Pacific Islander                         23 (0.2)        15 (0.3)         8 (0.2)        
                            None                                                              32 (0.3)        15 (0.3)        17 (0.4)        
                            Other                                                            248 (2.6)       136 (2.5)       112 (2.7)        
                            Unknown                                                          943 (9.8)      559 (10.3)       384 (9.1)        
                            White                                                          2510 (26.0)     1596 (29.3)      914 (21.6)        
ethnicity, n (%)            Hispanic                                                         636 (6.6)       364 (6.7)       272 (6.4)   0.034
                            Non-Hispanic                                                   8326 (86.1)     4652 (85.5)     3674 (87.0)        
                            None                                                              34 (0.4)        16 (0.3)        18 (0.4)        
                            Unknown                                                          673 (7.0)       412 (7.6)       261 (6.2)        
sex, n (%)                  Female                                                         3912 (40.5)     2253 (41.4)     1659 (39.3)   0.037
                            Male                                                           5757 (59.5)     3191 (58.6)     2566 (60.7)        
dbp, mean (SD)                                                                       0     65.7 (14.5)     65.3 (14.3)     66.1 (14.7)   0.007
heart_rate, mean (SD)                                                                0     89.6 (19.3)     89.8 (18.9)     89.3 (19.8)   0.146
height_cm, mean (SD)                                                                 0    153.1 (54.1)    151.7 (55.5)    154.8 (52.2)   0.004
map, mean (SD)                                                                       0     70.8 (31.5)     71.9 (30.0)     69.3 (33.2)  <0.001
respiratory_rate, mean (SD)                                                          0      19.7 (6.2)      20.0 (6.2)      19.4 (6.1)  <0.001
sbp, mean (SD)                                                                       0    120.0 (22.9)    119.8 (22.8)    120.3 (23.1)   0.298
spo2, mean (SD)                                                                      0      96.9 (4.7)      96.7 (4.5)      97.2 (4.8)  <0.001
temp_c, mean (SD)                                                                    0      36.8 (1.0)      36.8 (1.1)      36.8 (0.9)   0.489
weight_kg, mean (SD)                                                                 0     85.0 (27.4)     85.5 (26.1)     84.3 (29.1)   0.036
rass, mean (SD)                                                                    506      -1.9 (1.7)      -1.7 (1.6)      -2.1 (1.7)  <0.001
midazolam, mean (SD)                                              

In [59]:
gen_and_save_tableone(
    file_name='table_one_by_shift_hr72',
    data=cohort_results_72, 
    continuous=cont_vars, 
    categorical=['race', 'ethnicity', 'sex'], 
    groupby='shift',
    # nonnormal=['age', 'rass', 'sofa'] + resp_vars + vitals_vars,
    )

Grouped by shift                                                        
                                                                               Missing         Overall             day           night P-Value
n                                                                                                 3766            1983            1783        
age, mean (SD)                                                                       0     56.0 (17.8)     57.5 (17.3)     54.2 (18.2)  <0.001
race, n (%)                 American Indian or Alaska Native                                   8 (0.2)         7 (0.4)         1 (0.1)  <0.001
                            Asian                                                             51 (1.4)        25 (1.3)        26 (1.5)        
                            Black or African American                                      2283 (60.6)     1134 (57.2)     1149 (64.4)        
                            Native Hawaiian or Other Pacific Islander                          7 (0.2)         4 (0.2)         3 (0.2)        
                            None                                                              12 (0.3)         3 (0.2)         9 (0.5)        
                            Other                                                            118 (3.1)        60 (3.0)        58 (3.3)        
                            Unknown                                                          368 (9.8)      209 (10.5)       159 (8.9)        
                            White                                                           919 (24.4)      541 (27.3)      378 (21.2)        
ethnicity, n (%)            Hispanic                                                         282 (7.5)       152 (7.7)       130 (7.3)   0.018
                            Non-Hispanic                                                   3216 (85.4)     1676 (84.5)     1540 (86.4)        
                            None                                                              13 (0.3)         3 (0.2)        10 (0.6)        
                            Unknown                                                          255 (6.8)       152 (7.7)       103 (5.8)        
sex, n (%)                  Female                                                         1498 (39.8)      829 (41.8)      669 (37.5)   0.008
                            Male                                                           2268 (60.2)     1154 (58.2)     1114 (62.5)        
dbp, mean (SD)                                                                       0     68.4 (14.9)     68.4 (14.7)     68.3 (15.2)   0.806
heart_rate, mean (SD)                                                                0     89.5 (18.4)     89.8 (18.1)     89.0 (18.8)   0.188
height_cm, mean (SD)                                                                 0    165.0 (34.9)    163.4 (37.6)    166.8 (31.6)   0.003
map, mean (SD)                                                                       0     76.5 (31.8)     77.8 (31.2)     74.9 (32.3)   0.005
respiratory_rate, mean (SD)                                                          0      19.8 (5.9)      20.0 (5.8)      19.6 (5.9)   0.033
sbp, mean (SD)                                                                       0    125.1 (22.7)    124.1 (22.7)    126.3 (22.7)   0.003
spo2, mean (SD)                                                                      0      96.9 (4.0)      96.5 (4.3)      97.3 (3.5)  <0.001
temp_c, mean (SD)                                                                    0      36.8 (0.7)      36.8 (0.7)      36.9 (0.7)   0.015
weight_kg, mean (SD)                                                                 0     87.3 (27.5)     87.5 (27.0)     87.0 (28.0)   0.596
rass, mean (SD)                                                                    145      -1.8 (1.9)      -1.6 (1.8)      -2.1 (1.9)  <0.001
midazolam, mean (SD)                                              

## Regression

In [60]:
cohort_results_24.columns

Index(['row_id', 'hospitalization_id', 'date_hr', 'cohort_flag', 'hr', 'shift',
       'age', 'race', 'ethnicity', 'sex', 'hospitalization_id_1', 'date_hr_1',
       'cohort_flag_1', 'dbp', 'heart_rate', 'height_cm', 'map',
       'respiratory_rate', 'sbp', 'spo2', 'temp_c', 'weight_kg', 'rass',
       'midazolam', 'lorazepam', 'hydromorphone', 'fentanyl', 'propofol',
       'dexmedetomidine', 'ketamine', 'norepinephrine', 'epinephrine',
       'phenylephrine', 'dopamine', 'dobutamine', 'vasopressin', 'angiotensin',
       'fentanyl_eq', 'midazolam_eq', 'ne_eq', 'n_pressors', 'sofa', 'pao2',
       'fio2', 'p_f'],
      dtype='object')

In [ ]:
import statsmodels.formula.api as smf
import pandas as pd

def run_regression_analysis(cohort_hr, outcome_predictors):
    """
    Run regression analysis for sedation outcomes
    
    Parameters:
    cohort_hr (str): Either '24' or '72' to specify which cohort to use
    outcome_predictors (dict): Dictionary mapping outcomes to their predictors
                              e.g., {"outcome_1": ["predictor_1", "predictor_2"]}
    
    Returns:
    dict: Dictionary containing fitted regression models
    """
    
    # Map cohort_hr to corresponding dataframe
    if cohort_hr == '24':
        data = cohort_results_24
    elif cohort_hr == '72':
        data = cohort_results_72
    else:
        raise ValueError("cohort_hr must be either '24' or '72'")
    
    categorical_vars = ['shift', 'race', 'ethnicity', 'sex']
    results = {}

    # Get all unique predictors and outcomes for data cleaning
    all_predictors = set()
    all_outcomes = list(outcome_predictors.keys())
    
    for predictors in outcome_predictors.values():
        all_predictors.update(predictors)
    
    all_predictors = list(all_predictors)

    # Create a clean dataset by dropping rows with NA values in key variables
    regression_data = data.dropna(subset=all_predictors + all_outcomes)

    print(f"Original dataset size: {len(data)}")
    print(f"Dataset size after dropping NAs: {len(regression_data)}")

    # Run regression for each outcome
    for outcome, predictors in outcome_predictors.items():
        print(f"\n{'='*60}")
        print(f"Regression for {outcome} using formula API (cohort_hr={cohort_hr})")
        print('='*60)
        
        # Create formula string
        # C() indicates categorical variable
        categorical_terms = [f"C({p})" for p in predictors if p in categorical_vars]
        continuous_terms = [p for p in predictors if p not in categorical_vars]
        
        formula = f"{outcome} ~ " + " + ".join(categorical_terms + continuous_terms)
        
        # Fit model
        model = smf.ols(formula=formula, data=regression_data)
        results[outcome] = model.fit()
        
        print(results[outcome].summary())
        
    # Save regression coefficients and statistics as CSV
    for outcome in outcome_predictors.keys():
        # Extract coefficient table
        coef_df = pd.DataFrame({
            'coefficient': results[outcome].params,
            'std_err': results[outcome].bse,
            't_value': results[outcome].tvalues,
            'p_value': results[outcome].pvalues,
            'conf_int_lower': results[outcome].conf_int()[0],
            'conf_int_upper': results[outcome].conf_int()[1]
        })
        
        # Add model statistics
        model_stats = pd.DataFrame({
            'statistic': ['r_squared', 'adj_r_squared', 'f_statistic', 'f_pvalue', 'aic', 'bic', 'n_obs'],
            'value': [
                results[outcome].rsquared,
                results[outcome].rsquared_adj,
                results[outcome].fvalue,
                results[outcome].f_pvalue,
                results[outcome].aic,
                results[outcome].bic,
                results[outcome].nobs
            ]
        })
        
        # Save coefficient table
        coef_path = f'output/final/{site}_reg_coeff_{outcome}_hr{cohort_hr}_{timestamp}.csv'
        coef_df.to_csv(coef_path)
        
        # Save model statistics
        stats_path = f'output/final/{site}_reg_stats_{outcome}_hr{cohort_hr}_{timestamp}.csv'
        model_stats.to_csv(stats_path, index=False)
    
    print(f"\nResults saved to:")
    print(f"  CSV coefficients: {coef_path}")
    print(f"  CSV statistics: {stats_path}")
    
    return results

ols_predictors = [
        'shift', 
        'age', 
        'race', 'ethnicity', 'sex', 
        'dbp', 'map', 'sbp', 
        'height_cm', 'weight_kg',
        'heart_rate', 'respiratory_rate', 'spo2', 'temp_c', 
        'ne_eq', 'n_pressors', 
        'rass', 
        'sofa', 
        'p_f'
    ]

# Define outcome-predictor mappings
outcome_predictors = {
    'fentanyl_eq': ols_predictors,
    'midazolam_eq': ols_predictors,
    'propofol': ols_predictors
}

# Run analysis for both cohorts
results_24 = run_regression_analysis('24', outcome_predictors)
results_72 = run_regression_analysis('72', outcome_predictors)

Original dataset size: 9669
Dataset size after dropping NAs: 8230

Regression for fentanyl_eq using formula API (cohort_hr=24)
                            OLS Regression Results                            
Dep. Variable:            fentanyl_eq   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     12.80
Date:                Wed, 06 Aug 2025   Prob (F-statistic):           1.09e-55
Time:                        12:39:24   Log-Likelihood:                -75970.
No. Observations:                8230   AIC:                         1.520e+05
Df Residuals:                    8202   BIC:                         1.522e+05
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                                                           coef    std err         